## Sentence Generation from Keywords

1. During generation-rule acquisition, generation
rules for each keyword are automatically
acquired.
2. Candidate-text sentences are constructed
during candidate-text construction by applying
the rules acquired in the first
step. Each candidate-text sentence is represented
by a graph or dependency tree.
3. Candidate-text sentences are ranked according
to their scores assigned during evaluation.
The scores are calculated as a
probability estimated by using a keywordproduction
model and a language model
that are trained with a corpus.
4. The candidate-text sentence that maximizes
the score or the candidate-text sentences
whose scores are over a threshold
are selected as output. The system can
also output candidate-text sentences that
are ranked within the top N sentences.


headword: rightmost content word

content word: word whose part-of-speech is a verb, adjective,noun, demonstrative, adverb, conjunction, attribute,

interjection, or undefined word.

function word: everything else, formal nouns and auxiliary verbs “SURU (do)” and “NARU (become)”

In [1]:
import re

In [2]:
t = "打给 美国/澳洲/英国/加拿大 是多少钱? (<i>dǎgěi měiguó/àozhōu/yīngguó/jiānádà shì duōshǎo qián?)</i>"
hi = t.split(" (<i>")[1]
print(hi[:-5])

dǎgěi měiguó/àozhōu/yīngguó/jiānádà shì duōshǎo qián?


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests      

wiki = "http://wikitravel.org/en/Arabic_phrasebook"
req = requests.get(wiki)
soup = BeautifulSoup(req.text, "lxml")
    
#soup = BeautifulSoup(body, 'lxml') # Parse the HTML as a string
df = pd.DataFrame(columns=["en","ar"])
cat = ""


In [4]:
labels = []
for dl in soup.find_all('dl'):
    if dl.find_all('dt') and not dl.find_all('dl'):
        labels.append(dl)     

In [5]:
ens = [] 
ars = []

df = pd.DataFrame(columns=["en","ar"])
for label in labels:
    dt = label.find_all("dt")
    dd = label.find_all("dd")
    
    for d in dd:
        if "Good night" in d.text:
            dd.remove(d)
            print("removed 1")
        
    for t in dt:
        if "antunna" in t.text:
            dt.remove(t)
            print("removed 1")
        if "(two people)" in t.text:
            dt.remove(t)
            print("removed 1")
        if "momken tfaye" in t.text:
            dt.remove(t)
            print("removed 1")
        
    if len(dt) == len(dd):
        for i in range(len(dt)):
            df.loc[len(df)] = [dt[i].text.strip(), dd[i].text.strip()]
    else:
        print(len(dt),len(dd))
        print("failed")
        print(label)
        


removed 1
removed 1
removed 1
removed 1


In [6]:
df.to_csv("en_ar_phrases_raw.csv")

In [7]:
df

,en,ar
0,OPEN,مفتوح (maftūh)
1,CLOSED,مغلق (mughlaq)
2,ENTRANCE,دخول (dukhūl)
3,EXIT,خروج (khurūj)
4,PUSH,ادفع (idfa`)
5,PULL,اسحب (ishab)
6,TOILET,حمام (hammām)
7,MEN,رجال (rijāl)
8,WOMEN,سيدات (sayyidāt)
9,FORBIDDEN,ممنوع (mamnū`)


In [25]:
for i, row in df.iterrows():
    if row.en in row.ar:
        row.ar = row.ar.strip(row.en).strip(" .")
    if row.en[0:3] == "...":
        row.en = row.en[3:len(row.en)-1]
    if row.ar[0:3] == "...":
        temp = row.ar.split("(")
        row.ar = temp[0][3:len(row.ar)-1]
    
    brack = re.compile(r'\(.*\)')
    unders = re.compile(r'_+')
    eng = re.compile(r'[a-zA-Z0-9āīūēō\-.,~\'`?!\[\]]+')
    row.ar = brack.sub("", row.ar)
    row.ar = unders.sub("*", row.ar)
    row.ar = eng.sub("", row.ar)
    
    

In [26]:
df

,en,ar
0,OPEN,مفتوح
1,CLOSED,مغلق
2,ENTRANCE,دخول
3,EXIT,خروج
4,PUSH,ادفع
5,PULL,اسحب
6,TOILET,حمام
7,MEN,رجال
8,WOMEN,سيدات
9,FORBIDDEN,ممنوع


In [27]:
df.to_csv("cleaned_arabic.csv")

In [31]:
cndf = pd.read_csv("../clean_en_cn.tsv", sep="\t")

In [40]:
cndf= cndf.drop(['Unnamed: 0','category'], axis=1)

In [41]:
ardf = df.copy()
cndf = cndf.rename(columns={"english": "en", "chinese": "cn"})

In [44]:
result = pd.merge(ardf, cndf, how='outer', on=['en'])

In [45]:
result

,en,ar,cn
0,OPEN,مفتوح,NaN
1,CLOSED,مغلق,NaN
2,ENTRANCE,دخول,NaN
3,EXIT,خروج,NaN
4,PUSH,ادفع,NaN
5,PULL,اسحب,NaN
6,TOILET,حمام,NaN
7,MEN,رجال,NaN
8,WOMEN,سيدات,NaN
9,FORBIDDEN,ممنوع,NaN


In [ ]:
result.to_csv("cleaned_mer")